# Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import math
from matplotlib.animation import FuncAnimation
from functools import partial
import itertools
import matplotlib.animation as animation

# Class Vector with all functions


In [2]:
class Vector:

    # angle between 0 and 360
    # length positive float max value = 10
    # speed negative or positive float between -1 and 1
    def __init__(self, angle, length, speed) -> None:
        assert angle >= 0   and angle <= 360
        assert length <= 10 and length > 0
        assert speed >= -1  and speed <= 1
        self.angle  = angle
        self.length = length
        self.speed  = speed

        self.endpoint = self.getEndpoint()
        self.radius   = self.getRadius()

        self.origin = [0, 0]

    def getEndpoint(self) -> list:
        angle = self.angle * math.pi / 180; 
        x = self.length * math.sin(angle)
        y = self.length * math.cos(angle)
        return [x, y]

    def getRadius(self) -> float:
        return math.sqrt(self.endpoint[0]*self.endpoint[0] + self.endpoint[1]*self.endpoint[1])

    # FIXME speed has to be adjusted
    def getTimedEndpoint(self, time) -> list:
        fixedSpeed = 10
        angleIncrement = fixedSpeed * self.speed * time
        angle = ((self.angle + angleIncrement) % 360) * math.pi / 180; 
        x = self.length * math.sin(angle)
        y = self.length * math.cos(angle)
        return [x, y]
    
    def setOrigin(self, origin):
        self.origin = origin

# Class for a datapoint

In [ ]:
class Datapoint():

    def __init__(self, time) -> None:
        self.time = time
        self.points = None
        self.vectors = []

    def addVector(self, vector):
        self.vectors.append(vector)
    
    def setPoints(self, points):
        self.points = points

    def getTime(self):
        return self.time
    
    def getPoints(self):
        return self.points
    
    def getVectors(self):
        return self.vectors